In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    "atr",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2021,1,1),  test_end=datetime(2022,1,1),
    search_type='lgmm', n_trials=500, n_models=1
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"]    = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['search_type']} {cfg.get('search_subtype', '')} {cfg['label_method']}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=16.91s mem=408.25MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=17.23s mem=410.50MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=19.02s mem=409.81MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=19.90s mem=409.11MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=26.05s mem=405.54MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=30.57s mem=406.28MB 
[XAUUSD H1 both lgmm] modelo 0 trial 2/500 ins=-1.000000 oos=-1.000000 avg=16.64s mem=418.43MB 
[XAUUSD H1 both lgmm] modelo 0 trial 2/500 ins=-1.000000 oos=-1.000000 avg=17.41s mem=422.71MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=-1.000000 oos=-1.000000 avg=37.39s mem=497.95MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=0.145331 oos=0.019642 avg=38.81s mem=532.68MB 
[XAUUSD H1 both lgmm] modelo 0 trial 1/500